In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm import tqdm
from datetime import datetime
import json
from sklearn.preprocessing import LabelEncoder
import calendar
import spotipy
warnings.filterwarnings("ignore")

# Web scrapping data from Spotify Chart

## Get date needed for each week

format of links to download:
https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download
https://spotifycharts.com/regional/global/weekly/2016-12-30--2017-01-06/download

We see that all the dates that appear in the link are Fridays, each being one week apart

In [ ]:
#An example of Calendar package
x = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2025, month)
    week1 = mycal[1]
    week2 = mycal[2]
      
    if week1[calendar.MONDAY] != 0:
        auditday = week1[calendar.MONDAY]
    else: 
        auditday = week2[calendar.MONDAY] 
    print("%10s %2d" % (calendar.month_name[month], auditday))
    x.append("%10s %2d" % (calendar.month_name[month], auditday))

In [ ]:
# reference
c = calendar.TextCalendar(calendar.MONDAY)
str1 = c.formatmonth(2017,1)
print(str1)

In [ ]:
(calendar.monthcalendar(2017,1)[0][calendar.FRIDAY],calendar.monthcalendar(2017,1)[4][calendar.FRIDAY])
#calendar.monthcalendar(2017,1)[c(0,4)][calendar.FRIDAY]

In [ ]:
#All the fridays in 2017
Fridays17 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2017, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2017, month, mycal[week][calendar.FRIDAY])
            Fridays17.append((2017, month, mycal[week][calendar.FRIDAY]))
            
            
Fridays17[0]

In [ ]:
#All the fridays in 2018
Fridays18 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2018, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2018, month, mycal[week][calendar.FRIDAY])
            Fridays18.append((2018, month, mycal[week][calendar.FRIDAY]))


Fridays18[0][2]

In [ ]:
#fridays in 2019 (Jan to Aug)
Fridays19 = []

for month in range(1, 9):
    mycal = calendar.monthcalendar(2019, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2019, month, mycal[week][calendar.FRIDAY])
            Fridays19.append((2019, month, mycal[week][calendar.FRIDAY]))

Fridays19[2]

In [ ]:
#Fridays in 2016
Fridays16to19 = [(2016, 12, 23),(2016, 12, 30)]

#Compile all the fridays 2016-2019
Fridays16to19 = Fridays16to19 + Fridays17 + Fridays18 + Fridays19
Fridays16to19[0]

In [ ]:
#formatting friday dates: 2016-12-23--2016-12-30

Fridays_format = []

for i in range(0, len(Fridays16to19)):
    year = Fridays16to19[i][0]
    month = Fridays16to19[i][1]
    day = Fridays16to19[i][2]  
    
    if month < 10:
        month = str(0)+str(month)
    else:
        month = str(month)
        
    if day < 10:
        day = str(0)+str(day)
    else:
        day = str(day)
    
    #print(str(year)+"-"+month+"-"+day)
    Fridays_format.append(str(year)+"-"+month+"-"+day)
    
Fridays_format[0]

In [ ]:
#put all the dates into the format of download link: 
#https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download

####################IndexError: list index out of range
# for j in range(0, len(Fridays_format)):
#     str_p1 = "https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + "--"
#     if j != len(Fridays_format):
#         print(str_p1 + Fridays_format[j+1] + "/download")
        
# for j in range(0, len(Fridays_format)):
#     print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] 
#           + "--" + Fridays_format[j+1] + "/download")


j = 0
while(j<len(Fridays_format)-1):
    str_p1 = print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + 
                   "--" + Fridays_format[j+1] + "/download")
    j += 1

## Billboard Dataset

In [ ]:
import csv
import pandas as pd

billboard_df = pd.read_csv("~/desktop/spotify_top_tracks_analysis/data/billboard_lyrics_1964-2015.csv",
                   encoding='latin-1')
billboard_df.head()

#billboard_df.describe()

In [ ]:
billboard_df.iloc[[4],[4]]

In [ ]:
billboard_df

## spotipy

In [ ]:
#setting up spotipy
import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()
    
token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
                                   client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
#test 1: Shawn Mendes
name = 'Shawn Mendes'

results = sp.search(q='artist:' + name, type='artist')
print(results)

In [ ]:
#test 2: Senorita Shawn Mendes
track_name = 'Senorita'
artist = 'Shawn Mendes'

results = sp.search(q= track_name + ' ' + artist)
print(results)

In [ ]:
len(results['tracks']) #a list of the results
results['tracks']['items'][0] #single
results['tracks']['items'][1] #Shawn Mendes' Album

In [ ]:
results['tracks']['items'][0].keys()

In [ ]:
track_id = results['tracks']['items'][0]['id']
track_id

In [ ]:
track = sp.audio_analysis('0TK2YIli7K1leLovkQiNik')
track

In [ ]:
track = sp.audio_features('0TK2YIli7K1leLovkQiNik')
track

### Get_Track_ID function

In [ ]:
def get_track_id(track_name = "", artist = ""):
    print(track_name, artist)
    q = track_name + ' ' + artist
    print(q)
    search_results = sp.search(q)
    track_id = search_results['tracks']['items'][0]['id']
    return track_id

In [ ]:
#Solving error 1 - Index Error
def get_track_id(track_name = "", artist = ""):
    print(track_name, artist)
    q = track_name + ' ' + artist
    print(q)
    search_results = sp.search(q)
    try:
        track_id = search_results['tracks']['items'][0]['id']
    except IndexError:
        track_id = 'null'
    return track_id

In [ ]:
#Solving error 2 - 'featuring'
def get_track_id(track_name = "", artist = ""):
    # 'featuring' problem
    if ('featuring' in artist) == True:
        artist = artist.split(' featuring')[0] + artist.split(' featuring')[1]
    else:
        artist = artist
    print(track_name, artist)
    
    q = track_name + ' ' + artist
    search_results = sp.search(q)
    
    #Index Error
    try:
        track_id = search_results['tracks']['items'][0]['id']
    except IndexError:
        track_id = 'null'
    return track_id

In [ ]:
#sample of first 5 rows with track ID
sample_df = billboard_df.iloc[:5, :]
sample_df

In [ ]:
sample_df['Spotify_Track_ID'] = sample_df.apply(lambda row: get_track_id(row['Song'],row['Artist']), axis=1)



In [ ]:
sample_df.to_csv('sample.csv')

In [ ]:
get_track_id('wooly bully','sam the sham and the pharaohs')

In [ ]:
billboard_df.iloc[[0],[2]] #in the form of data frame
billboard_df.iloc[0,1] #in the form of string
get_track_id(billboard_df.iloc[0,1],billboard_df.iloc[0,2])

In [ ]:
billboard_df['Spotify_Track_ID'] = billboard_df.apply(lambda row: get_track_id(row['Song'],row['Artist']), axis=1)
#axis lets apply function know which way it is going through (by row or by column)

In [ ]:
billboard_df.Spotify_Track_ID.value_counts() #256 nulls after removing 'featuring'

In [ ]:
value_list = ['null']
billboard_df[billboard_df.Spotify_Track_ID.isin(value_list)].Year.value_counts()

In [ ]:
value_list = ['null']
billboard_df[billboard_df.Spotify_Track_ID.isin(value_list)]
#billboard_df[billboard_df.Spotify_Track_ID.isin(value_list)].Song.value_counts()

In [ ]:
get_track_id('bad blood taylor swift kendrick lamar')

In [ ]:
get_track_id('lean on major lazer')

In [ ]:
sp.track('1Lim1Py7xBgbAkAys3AGAG')

In [ ]:
# Second error - "featuring" in Artist cause a big problem
#billboard_df.iloc[5015,:]
('featuring' in billboard_df.iloc[5015,2])
#billboard_df.iloc[5015,2][:billboard_df.iloc[5015,2].index("featuring")]

In [ ]:
a = billboard_df.iloc[5014,2].split(' featuring')[0] + billboard_df.iloc[5014,2].split(' featuring')[1]
a

In [ ]:
if ('featuring' in billboard_df.iloc[5014,2]) == True:
    print(billboard_df.iloc[5014,2].split(' featuring')[0] + billboard_df.iloc[5014,2].split(' featuring')[1])
else:
    print(billboard_df.iloc[5014,2])

In [ ]:
# First error - IndexError: list index out of range
billboard_df.iloc[131]
get_track_id(billboard_df.iloc[131,1],billboard_df.iloc[131,2])

In [ ]:
billboard_df.iloc[[131],[1]] + " " + billboard_df.iloc[[131],[2]] 
#NaN because of the data structure

In [ ]:
get_track_id(billboard_df.iloc[[131],[1]], billboard_df.iloc[[131],[2]]) 
#NaN somehow has a default id '1f5cbQtDrykjarZVrShaDI'
get_track_id(billboard_df.iloc[[0],[1]], billboard_df.iloc[[0],[2]])

In [ ]:
billboard_dfrd_dfboard_df.head()

In [ ]:
sp.track('5tyMJlMqaggzvuX7TtlrTe')

### Sample 10000 and get features

In [12]:
billboard_with_id_df = pd.read_csv("~/desktop/spotify_top_tracks_analysis/data/billboard_with_ids.csv")

In [15]:
billboard_with_id_df.shape

(290913, 13)

In [16]:
sample = billboard_with_id_df.sample(n=10000, random_state=2019)

In [19]:
import os
path='~/desktop/spotify_top_tracks_analysis/data/'
sample.to_csv(os.path.join(path,r'billboard_with_ids_10000_sample.csv'))

#sample.to_csv('billboard_with_ids_10000_sample.csv')